# Advanced Federated Learning for Cross-Bank Fraud Detection

## With Differential Privacy, FedDANE, and Privacy Auditing

This notebook demonstrates an enhanced federated learning framework with:
- **FedAvg**: Standard federated averaging
- **FedProx**: Improved optimization with proximal regularization
- **FedDANE**: Variance-reduced aggregation (NEW)
- **Differential Privacy (DP-SGD)**: Privacy-preserving training (NEW)
- **Privacy Auditing**: Membership inference attacks and privacy metrics (NEW)
- **Heterogeneity Simulation**: Non-IID data and client dropout (NEW)

## Step 1: Install and Import Dependencies

In [ ]:
# Install required packages
import subprocess
import sys

packages = ['torch', 'pandas', 'scikit-learn', 'numpy', 'matplotlib', 'seaborn']
for package in packages:
    try:
        __import__(package)
    except ImportError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', package])

print('✅ All packages installed')

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import seaborn as sns
import os
from typing import List, Dict, Tuple
import warnings

warnings.filterwarnings('ignore')

# Import custom modules
import sys
sys.path.insert(0, '..')

from federated_learning.models import FraudDetectionModel, FraudDetectionModelEnhanced
from federated_learning.privacy import DifferentialPrivacyEngine, MembershipInferenceAttack
from federated_learning.aggregators import FedAvgAggregator, FedProxAggregator, FedDANEAggregator
from federated_learning.utils import DataPreprocessor
from federated_learning.utils.training import ClientTrainer, ModelEvaluator, TrainingMetricsTracker

# Setup device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'🖥️ Using device: {device}')

# Set seeds for reproducibility
torch.manual_seed(42)
np.random.seed(42)

## Step 2: Load and Preprocess Data

In [ ]:
# Configure data paths
data_dir = '../Data'
csv_files = [os.path.join(data_dir, f) for f in os.listdir(data_dir) if f.endswith('.csv')]

print(f'📁 Found {len(csv_files)} CSV files:')
for f in csv_files:
    print(f'  - {os.path.basename(f)}')

# Load and preprocess data
preprocessor = DataPreprocessor()
client_data, input_dim = preprocessor.load_and_preprocess_csvs(
    csv_files,
    label_column='Is_Fraud',
    test_size=0.2,
    random_state=42
)

print(f'\n✅ Data loaded and preprocessed')
print(f'📊 Number of clients: {len(client_data)}')
print(f'🔢 Input feature dimension: {input_dim}')
print(f'📋 Feature names: {list(preprocessor.feature_names)}')

# Print data distribution
for i, (train_df, test_df) in enumerate(client_data, 1):
    fraud_rate = train_df['Is_Fraud'].mean()
    print(f'\nClient {i}: Train={len(train_df)}, Test={len(test_df)}, Fraud Rate={fraud_rate:.4f}')

## Step 3: Create DataLoaders

In [ ]:
# Create DataLoaders for each client
batch_size = 32
client_train_loaders = []
client_test_loaders = []

for i, (train_df, test_df) in enumerate(client_data):
    train_loader, test_loader = preprocessor.create_dataloaders(
        train_df, test_df,
        label_column='Is_Fraud',
        batch_size=batch_size
    )
    client_train_loaders.append(train_loader)
    client_test_loaders.append(test_loader)

print(f'✅ Created {len(client_train_loaders)} train and test loaders')

## Step 4: Compare Federated Learning Algorithms

In [ ]:
# Configuration for federated learning
num_rounds = 5
local_epochs = 2
learning_rate = 0.001

algorithms = {
    'FedAvg': {'aggregator': FedAvgAggregator(), 'mu': 0.0},
    'FedProx (μ=0.01)': {'aggregator': FedProxAggregator(mu=0.01), 'mu': 0.01},
    'FedDANE': {'aggregator': FedDANEAggregator(learning_rate=0.01), 'mu': 0.0},
}

results = {}
evaluator = ModelEvaluator(device=device)

for alg_name, alg_config in algorithms.items():
    print(f'\n{"="*60}')
    print(f'🚀 Training with {alg_name}')
    print(f'{"="*60}')
    
    aggregator = alg_config['aggregator']
    mu = alg_config['mu']
    
    # Initialize global model
    global_model = FraudDetectionModel(input_dim).to(device)
    trainer = ClientTrainer(model=None, device=device, learning_rate=learning_rate)
    
    client_accuracies = {i: [] for i in range(len(client_data))}
    round_accuracies = []
    
    # Federated training rounds
    for round_num in range(num_rounds):
        print(f'\n🔁 Round {round_num + 1}/{num_rounds}')
        
        client_models = []
        
        # Local training on each client
        for client_id, train_loader in enumerate(client_train_loaders):
            # Create client model with global weights
            client_model = FraudDetectionModel(input_dim).to(device)
            client_model.load_state_dict(global_model.state_dict())
            
            # Train client
            trainer.model = client_model
            train_metrics = trainer.train_one_round(
                train_loader,
                epochs=local_epochs,
                global_model=global_model if mu > 0 else None,
                mu=mu
            )
            
            client_models.append(client_model)
        
        # Server aggregation
        if isinstance(aggregator, FedDANEAggregator):
            aggregator.aggregate(client_models, global_model, client_updates=None)
        else:
            aggregator.aggregate(client_models, global_model)
        
        # Evaluate on test set
        round_acc = []
        for client_id, test_loader in enumerate(client_test_loaders):
            metrics = evaluator.evaluate(global_model, test_loader, label=f'Client {client_id+1} (Round {round_num+1})')
            acc = metrics['accuracy']
            client_accuracies[client_id].append(acc)
            round_acc.append(acc)
        
        avg_round_acc = np.mean(round_acc)
        round_accuracies.append(avg_round_acc)
        print(f'📈 Average accuracy: {avg_round_acc:.4f}')
    
    results[alg_name] = {
        'client_accuracies': client_accuracies,
        'round_accuracies': round_accuracies,
        'final_accuracies': [accs[-1] for accs in client_accuracies.values()]
    }

print(f'\n✅ All algorithms trained successfully!')

## Step 5: Compare Results

In [ ]:
# Create comparison table
print(f'\n{"="*70}')
print('📊 FINAL ACCURACY COMPARISON TABLE')
print(f'{"="*70}')

comparison_df = pd.DataFrame()
for alg_name, result in results.items():
    comparison_df[alg_name] = result['final_accuracies']

comparison_df.index = [f'Client {i+1}' for i in range(len(client_data))]
print(comparison_df.to_string())
print(f'\nAverage: {comparison_df.mean().to_dict()}')
print(f'{"="*70}')

## Step 6: Convergence Analysis

In [ ]:
# Plot convergence curves
plt.figure(figsize=(14, 6))

for i, (alg_name, result) in enumerate(results.items()):
    rounds = list(range(1, num_rounds + 1))
    plt.plot(rounds, result['round_accuracies'], marker='o', label=alg_name, linewidth=2, markersize=8)

plt.xlabel('Communication Round', fontsize=12)
plt.ylabel('Average Accuracy', fontsize=12)
plt.title('Federated Learning Algorithm Convergence Comparison', fontsize=14, fontweight='bold')
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print('✅ Convergence plot generated')

## Step 7: Differential Privacy Training

In [ ]:
print(f'\n{"="*60}')
print('🔐 DIFFERENTIAL PRIVACY FEDERATED LEARNING')
print(f'{"="*60}')

# DP Configuration
dp_configs = {
    'No Privacy': {'noise_multiplier': 0.0, 'max_grad_norm': 1.0},
    'DP-SGD (σ=0.5)': {'noise_multiplier': 0.5, 'max_grad_norm': 1.0},
    'DP-SGD (σ=1.0)': {'noise_multiplier': 1.0, 'max_grad_norm': 1.0},
}

dp_results = {}

for dp_name, dp_config in dp_configs.items():
    print(f'\n🚀 Training with {dp_name}')
    print(f'  Noise Multiplier: {dp_config["noise_multiplier"]}')
    
    # Initialize privacy engine
    privacy_engine = DifferentialPrivacyEngine(
        noise_multiplier=dp_config['noise_multiplier'],
        max_grad_norm=dp_config['max_grad_norm'],
        delta=1e-5
    )
    
    # Initialize global model
    global_model = FraudDetectionModel(input_dim).to(device)
    trainer = ClientTrainer(model=None, device=device, learning_rate=learning_rate)
    aggregator = FedAvgAggregator()
    
    round_accuracies = []
    privacy_budgets = []
    
    # Federated training with DP
    for round_num in range(num_rounds):
        client_models = []
        
        for train_loader in client_train_loaders:
            client_model = FraudDetectionModel(input_dim).to(device)
            client_model.load_state_dict(global_model.state_dict())
            
            trainer.model = client_model
            trainer.train_one_round(
                train_loader,
                epochs=local_epochs,
                use_dp=True,
                dp_engine=privacy_engine
            )
            
            client_models.append(client_model)
        
        aggregator.aggregate(client_models, global_model)
        
        # Evaluate
        accs = []
        for test_loader in client_test_loaders:
            metrics = evaluator.evaluate(global_model, test_loader, label=f'DP-{dp_name} Round {round_num+1}')
            accs.append(metrics['accuracy'])
        
        avg_acc = np.mean(accs)
        round_accuracies.append(avg_acc)
        
        # Compute privacy loss
        total_samples = sum(len(loader.dataset) for loader in client_train_loaders)
        epsilon, delta = privacy_engine.compute_privacy_loss_basic(
            total_samples, batch_size, round_num + 1
        )
        privacy_budgets.append((epsilon, delta))
        
        print(f'  Round {round_num + 1}: Acc={avg_acc:.4f}, ε={epsilon:.4f}, δ={delta}')
    
    dp_results[dp_name] = {
        'accuracies': round_accuracies,
        'privacy_budgets': privacy_budgets
    }

print(f'\n✅ Differential Privacy training complete!')

## Step 8: Privacy-Utility Trade-off Analysis

In [ ]:
# Plot privacy-utility trade-off
plt.figure(figsize=(12, 5))

# Extract epsilon and accuracy for each configuration
epsilons_by_config = {}
accuracies_by_config = {}

for dp_name, result in dp_results.items():
    epsilons = [budget[0] for budget in result['privacy_budgets']]
    accuracies = result['accuracies']
    epsilons_by_config[dp_name] = epsilons
    accuracies_by_config[dp_name] = accuracies

# Plot 1: Accuracy over rounds
plt.subplot(1, 2, 1)
for dp_name, accs in accuracies_by_config.items():
    plt.plot(range(1, num_rounds + 1), accs, marker='o', label=dp_name, linewidth=2)
plt.xlabel('Round', fontsize=11)
plt.ylabel('Accuracy', fontsize=11)
plt.title('Model Accuracy with Different Privacy Levels', fontsize=12, fontweight='bold')
plt.legend()
plt.grid(True, alpha=0.3)

# Plot 2: Privacy-Utility Trade-off
plt.subplot(1, 2, 2)
for dp_name in dp_results.keys():
    eps = epsilons_by_config[dp_name]
    accs = accuracies_by_config[dp_name]
    plt.plot(eps, accs, marker='s', label=dp_name, linewidth=2, markersize=7)
plt.xlabel('Privacy Budget (ε)', fontsize=11)
plt.ylabel('Accuracy', fontsize=11)
plt.title('Privacy-Utility Trade-off', fontsize=12, fontweight='bold')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print('✅ Privacy-Utility trade-off plot generated')

## Step 9: Non-IID Data Distribution and Heterogeneity Analysis

In [ ]:
print(f'\n{"="*60}')
print('📊 NON-IID DATA HETEROGENEITY SIMULATION')
print(f'{"="*60}')

# Combine all data for non-IID split
all_train_data = pd.concat([train_df for train_df, _ in client_data], ignore_index=True)

# Create non-IID distributions
iid_degrees = [1.0, 0.5, 0.1]  # 1.0 = IID, 0.0 = fully non-IID
heterogeneity_results = {}

for iid_degree in iid_degrees:
    print(f'\n🚀 Training with IID Degree = {iid_degree}')
    
    # Create non-IID splits
    non_iid_clients = preprocessor.create_non_iid_data_split(
        all_train_data,
        num_clients=len(client_data),
        iid_degree=iid_degree,
        seed=42
    )
    
    # Create loaders for non-IID data
    non_iid_train_loaders = []
    non_iid_test_loaders = []
    
    for client_df in non_iid_clients:
        if len(client_df) > 0:
            train_df = client_df.sample(frac=0.8, random_state=42)
            test_df = client_df.drop(train_df.index)
            
            train_loader, test_loader = preprocessor.create_dataloaders(
                train_df, test_df,
                label_column='Is_Fraud',
                batch_size=batch_size
            )
            non_iid_train_loaders.append(train_loader)
            non_iid_test_loaders.append(test_loader)
    
    # Train with FedAvg on non-IID data
    global_model = FraudDetectionModel(input_dim).to(device)
    trainer = ClientTrainer(model=None, device=device, learning_rate=learning_rate)
    aggregator = FedAvgAggregator()
    
    round_accuracies = []
    
    for round_num in range(num_rounds):
        client_models = []
        for train_loader in non_iid_train_loaders:
            client_model = FraudDetectionModel(input_dim).to(device)
            client_model.load_state_dict(global_model.state_dict())
            
            trainer.model = client_model
            trainer.train_one_round(train_loader, epochs=local_epochs)
            
            client_models.append(client_model)
        
        aggregator.aggregate(client_models, global_model)
        
        # Evaluate
        accs = []
        for test_loader in non_iid_test_loaders:
            metrics = evaluator.evaluate(global_model, test_loader, label=f'Non-IID (IID%={iid_degree*100}) Round {round_num+1}')
            accs.append(metrics['accuracy'])
        
        avg_acc = np.mean(accs)
        round_accuracies.append(avg_acc)
        print(f'  Round {round_num + 1}: Avg Accuracy = {avg_acc:.4f}')
    
    heterogeneity_results[f'IID%={iid_degree*100}'] = round_accuracies

print(f'\n✅ Heterogeneity simulation complete!')

## Step 10: Client Dropout Simulation

In [ ]:
print(f'\n{"="*60}')
print('🔌 CLIENT DROPOUT SIMULATION')
print(f'{"="*60}')

dropout_rates = [0.0, 0.2, 0.4]
dropout_results = {}

for dropout_rate in dropout_rates:
    print(f'\n🚀 Training with Dropout Rate = {dropout_rate*100}%')
    
    global_model = FraudDetectionModel(input_dim).to(device)
    trainer = ClientTrainer(model=None, device=device, learning_rate=learning_rate)
    aggregator = FedAvgAggregator()
    
    round_accuracies = []
    
    for round_num in range(num_rounds):
        # Simulate client dropout
        active_clients = preprocessor.simulate_client_dropout(
            len(client_train_loaders),
            dropout_rate=dropout_rate,
            seed=round_num
        )
        
        num_active = np.sum(active_clients)
        print(f'  Round {round_num + 1}: {num_active}/{len(active_clients)} clients active')
        
        client_models = []
        active_train_loaders = [loader for loader, active in zip(client_train_loaders, active_clients) if active]
        
        for train_loader in active_train_loaders:
            client_model = FraudDetectionModel(input_dim).to(device)
            client_model.load_state_dict(global_model.state_dict())
            
            trainer.model = client_model
            trainer.train_one_round(train_loader, epochs=local_epochs)
            
            client_models.append(client_model)
        
        aggregator.aggregate(client_models, global_model)
        
        # Evaluate on all test clients
        accs = []
        for test_loader in client_test_loaders:
            metrics = evaluator.evaluate(global_model, test_loader, label=f'Dropout={dropout_rate*100}% Round {round_num+1}')
            accs.append(metrics['accuracy'])
        
        avg_acc = np.mean(accs)
        round_accuracies.append(avg_acc)
        print(f'    Avg Accuracy = {avg_acc:.4f}')
    
    dropout_results[f'Dropout {dropout_rate*100}%'] = round_accuracies

print(f'\n✅ Dropout simulation complete!')

## Step 11: Heterogeneity and Robustness Analysis

In [ ]:
# Plot heterogeneity and dropout effects
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Plot 1: IID vs Non-IID
for label, accs in heterogeneity_results.items():
    axes[0].plot(range(1, num_rounds + 1), accs, marker='o', label=label, linewidth=2)
axes[0].set_xlabel('Round', fontsize=11)
axes[0].set_ylabel('Accuracy', fontsize=11)
axes[0].set_title('Effect of Data Heterogeneity (Non-IID)', fontsize=12, fontweight='bold')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Plot 2: Client Dropout
for label, accs in dropout_results.items():
    axes[1].plot(range(1, num_rounds + 1), accs, marker='s', label=label, linewidth=2)
axes[1].set_xlabel('Round', fontsize=11)
axes[1].set_ylabel('Accuracy', fontsize=11)
axes[1].set_title('Effect of Client Dropout', fontsize=12, fontweight='bold')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print('✅ Heterogeneity and robustness plots generated')

## Step 12: Summary and Insights

In [ ]:
print(f'\n{"="*70}')
print('📈 COMPREHENSIVE FEDERATED LEARNING ANALYSIS SUMMARY')
print(f'{"="*70}')

print('\n1️⃣ ALGORITHM COMPARISON (FedAvg vs FedProx vs FedDANE):')
print('-' * 70)
for alg_name, result in results.items():
    final_acc = np.mean(result['final_accuracies'])
    print(f'  {alg_name}: Final Accuracy = {final_acc:.4f}')

print('\n2️⃣ PRIVACY-UTILITY TRADE-OFF:')
print('-' * 70)
for dp_name, result in dp_results.items():
    final_acc = result['accuracies'][-1]
    final_eps = result['privacy_budgets'][-1][0]
    print(f'  {dp_name}: Accuracy={final_acc:.4f}, ε={final_eps:.4f}')

print('\n3️⃣ ROBUSTNESS TO DATA HETEROGENEITY:')
print('-' * 70)
for label, accs in heterogeneity_results.items():
    print(f'  {label}: Final Accuracy = {accs[-1]:.4f}')

print('\n4️⃣ ROBUSTNESS TO CLIENT DROPOUT:')
print('-' * 70)
for label, accs in dropout_results.items():
    print(f'  {label}: Final Accuracy = {accs[-1]:.4f}')

print(f'\n{"="*70}')
print('✅ Analysis Complete!')
print(f'{"="*70}')